## SQL-zipcode

In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
webExtract = pd.read_csv("data/nyc_inspection_data/WebExtract.txt").fillna(0)

/Users/yutongpang/.virtualenvs/sqlVen27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(webExtract.CAMIS.unique())

25232

In [4]:
webExtract["INSPDATE"] = pd.to_datetime(webExtract.INSPDATE)

In [5]:
webExtract = webExtract.sort_values(by=["INSPDATE","SCORE"])

In [6]:
webExtract = webExtract.groupby("CAMIS").last()

In [7]:
webExtract['CAMIS'] = webExtract.index

In [10]:
zipcodegroup = webExtract.groupby("ZIPCODE").agg({'CAMIS':{"number":lambda x: x.nunique()},"SCORE":{"mean":np.mean,"std":np.std}})

In [11]:
zipcodegroup.head()

SCORE             CAMIS
              std       mean number
ZIPCODE                            
0.0      0.000000   0.000000      2
7005.0        NaN  11.000000      1
10000.0       NaN  13.000000      1
10001.0  6.915606   8.697446    509
10002.0  8.657194  10.896624    474

In [9]:
zipcodegroup.columns = zipcodegroup.columns.droplevel(0)

In [10]:
zipcodegroup = zipcodegroup[zipcodegroup.number>100]

In [11]:
len(zipcodegroup)

92

In [12]:
zipcodegroup['std'] = zipcodegroup['std']/(zipcodegroup.number)**0.5

In [13]:
len(zipcodegroup)

92

In [14]:
zipcodegroup = zipcodegroup.sort_values(by="mean")

In [15]:
zipcode = map(lambda x1,x2,x3,x4: (str(int(x1)),x2,x3,x4) ,zipcodegroup.index, zipcodegroup['mean'], zipcodegroup['std'], zipcodegroup.number)

In [16]:
zipcode

[('10001', 8.697445972495089, 0.30652883825886379, 509),
 ('10451', 8.7834394904458595, 0.54116200724977692, 157),
 ('10452', 8.8415841584158414, 0.61301602603174077, 101),
 ('10004', 8.9929577464788739, 0.54453968823957088, 142),
 ('10007', 9.2302158273381298, 0.53296272835119662, 139),
 ('11236', 9.2432432432432439, 0.72908637882157956, 111),
 ('11234', 9.3947368421052637, 0.52342794348371613, 152),
 ('11430', 9.4932432432432439, 0.72454007586308189, 148),
 ('11207', 9.625, 0.63648489175250167, 136),
 ('11209', 9.6590909090909083, 0.52817437737433648, 264),
 ('11231', 9.6603773584905657, 0.53674992755366324, 159),
 ('11238', 9.6626016260162597, 0.5988900182336705, 246),
 ('11217', 9.6889763779527556, 0.43586578068504855, 254),
 ('10472', 9.7407407407407405, 0.64292759823177648, 108),
 ('10306', 9.7477477477477485, 0.49658431482590842, 111),
 ('11101', 9.7520000000000007, 0.45169247099482807, 250),
 ('11218', 9.8027210884353746, 0.68243668298044802, 147),
 ('11201', 9.8173913043478258

In [17]:
import pickle

In [18]:
with open('zipcode.pickle', 'wb') as f:
    pickle.dump(zipcode, f)

In [20]:
zipcodegroup['ZIPCODE'] = zipcodegroup.index

In [22]:
zipcodegroup.index = range(len(zipcodegroup))

In [31]:
zipcodegroup['ZIPCODE'] = zipcodegroup.ZIPCODE.apply(lambda x: int(x))

In [33]:
zipcodegroup[["ZIPCODE","mean"]].to_csv("zipcode.csv")